In [13]:
import pandas as pd
autentica = pd.read_excel('ls.xlsx')

display(autentica)


login = str(autentica['login'][2])
senha = str(autentica['senha'][2])

indisponivel = "indisponivel"

,login,senha
0,2184579,sis1986%
1,2418048,71593
2,3216787,Hugo3428


In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time

def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [15]:
df_elemento_despesa = pd.DataFrame()
df_temp = pd.DataFrame(columns=('value', 'ed'))
df_temp.loc[0] = ['14', 'esportes']
# display(df_temp)

opcoes = []
valores = []

def varrer_ed():
#     abrir_suap()
    nav.get("https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/")
        
    valor_ed = nav.find_element_by_id('id_elemento_despesa')
    options = [x for x in valor_ed.find_elements_by_tag_name('option')]
    
    for i in options:
        # print(i.get_attribute('value')+" "+i.text)
        opcoes.append(i.text)
        valores.append(i.get_attribute('value'))
#         print(f'{i.text}-{i.get_attribute("value")}')

    df_elemento_despesa['value'] = valores
    df_elemento_despesa['ed']= opcoes
    
#     display(df_elemento_despesa)

In [16]:
def get_qtd():
    
    abrir_suap()
    
    df_tabela = pd.DataFrame()
    tabela = []
    codigos = []
    descricoes = []
    quantidades = []
    valores = []
    elemento = []
    
    try:
        pag = 1
        saida = True
        while saida:
                
            nav.get(f'https://suap.ifpe.edu.br/almoxarifado/form_busca/materialconsumo/?page={pag}&material=&elemento_despesa=&estoque_tipo=emcampus&unidade_medida=')
            celula = nav.find_element_by_tag_name('td')
                
            if celula:
                lista = nav.find_element_by_tag_name('table')
                td = lista.find_elements_by_tag_name('td')
                
                for i in td:
                    tabela.append(i.text)
                
                pag += 1
            else:
                saida = False
                
    except NoSuchElementException:
        print('acabaram as páginas')
        
    for cod in range(0, len(tabela), 7):
        codigos.append(tabela[cod])

    for des in range(1, len(tabela), 7):
        descricoes.append(tabela[des])

    for qtd in range(4, len(tabela), 7):
        quantidades.append(tabela[qtd])
    
    for val in range(5, len(tabela), 7):
        valores.append(tabela[val])
        
    for ed in range(3, len(tabela), 7):
        elemento.append(tabela[ed])

    df_tabela['codigo'] = codigos
    df_tabela['descricao'] = descricoes
    
    df_tabela['qtd'] = quantidades
    df_tabela['qtd'] = pd.to_numeric(df_tabela['qtd'])
    
    df_tabela['vlr_total'] = valores
    df_tabela['vlr_total'] = df_tabela['vlr_total'].str[2:]
    df_tabela['vlr_total'] = pd.to_numeric(df_tabela['vlr_total'])
    
    df_tabela['vlr_unt'] = df_tabela['vlr_total'] / df_tabela['qtd']
    df_tabela['vlr_unt'] = df_tabela['vlr_unt'].round(2)
    
    df_tabela['ed'] = elemento
    
    df_tabela.to_excel('relatorio/relatorio.xlsx', index=False)
    display(df_tabela)
    
    nav.quit()


In [17]:
get_qtd()

acabaram as páginas


,codigo,descricao,qtd,vlr_total,vlr_unt,ed
0,30.11.504,"FERRICIANETO DE POTÁSSIO, ASPECTO FÍSICO PÓ CR...",2,228.00,114.00,11 - MATERIAL QUIMICO
1,30.11.935,"CITRATO DE FERRO III E AMÔNIO, KG",2,537.98,268.99,11 - MATERIAL QUIMICO
2,30.11.941,"CARBONATO DE CÁLCIO, PÓ, FRASCO 500G",1,150.00,150.00,11 - MATERIAL QUIMICO
3,30.16.1026,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",4,230.52,57.63,16 - MATERIAL DE EXPEDIENTE
4,30.16.1044,"FITA ADESIVA, CREPE MONOFACE, 25MM X 50M.",22,71.72,3.26,16 - MATERIAL DE EXPEDIENTE
...,...,...,...,...,...,...
419,30.42.932,CHAVE TESTE MEDIDORA DIGITAL DE VOLTAGEM 12-220V.,10,66.60,6.66,42 - FERRAMENTAS
420,30.44.69,PLACA DE SINALIZAÇÃO DE AMBIENTE,6,129.00,21.50,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
421,30.50.33,"BANDEIRA DE MESA COM BRASÃO DO IFPE, MEDINDO 1...",3,34.77,11.59,"50 - BANDEIRAS, FLAMULAS E INSIGNIAS"
422,30.50.43,"BANDEIRA DO INSTITUTO FEDERAL DE PERNAMBUCO, D...",1,129.90,129.90,"50 - BANDEIRAS, FLAMULAS E INSIGNIAS"
